# Week 1 Lecture 2 - The garden of forking data


Throughout the course we will be using these [Jupyter notebooks](https://jupyter.org/) to develop, run, and share Python code. They work with R also.

McElreath's lectures for the whole book are available here: https://github.com/rmcelreath/stat_rethinking_2022

An excellent port to Python/PyMC Code is available here: https://github.com/dustinstansbury/statistical-rethinking-2023

You are encouraged to work through both of these versions to re-enforce what we're doing in class.

In [ ]:
# Import python packages
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
import random as rd
import pymc as pm

# Grid approximation

The best way to understand how Bayes theorem works is to work through it by hand - the grid approximation is a good way to do this because you can see visually how parameters and data interact through the likelhood. It also gives a sense of how priors become posteiors.

We'll use the globe tossing example because it conveys so well how probability can be built up and used from scratch.

In [ ]:
# Define grid
p_grid = np.linspace(0,1,20)
p_grid

In [ ]:
# Define prior
prior = np.ones(20)
prior

In [ ]:
# Normalize
prior = prior/sum(prior)
prior1 = prior
prior

Next we need to define the likelihood which, because it is descrete, is given by the probability mass function (PMF)

$$
\binom{n}{k}p^{x}(1-p)^{n-x}
$$

where

$$
\binom{n}{k} = \frac{n!}{k!(n - k)!}
$$

which in code is

In [ ]:
from math import factorial as f

# Binomial distribution
def dbinom(x,n,p):
    return f(n)/(f(x)*f(n-x))*p**(x)*(1-p)**(n-x)

# Data loop

In [ ]:
# Plot prior over range of p_grid
plt.plot(p_grid, prior)
plt.xlabel('Proportion water'),plt.ylabel('Plausability');

In [ ]:
# New observations
W = 0
L = 1
# Number of trials
N = W+L

# Calculate likelihood
likelihood = dbinom(W,N,p_grid)
likelihood

In [ ]:
# Plot likelihood over range of p_grid
plt.plot(p_grid, likelihood)
plt.xlabel('Proportion water'),plt.ylabel('Likelihood');

In [ ]:
sum(likelihood)

In [ ]:
likelihood*prior

In [ ]:
# Normalizing constant
sum(likelihood*prior)

In [ ]:
# Bayes theorem
posterior = (likelihood*prior)/sum(likelihood*prior)
posterior

In [ ]:
# Plot posterior over range of p_grid
plt.plot(p_grid, posterior)
plt.plot(p_grid, prior,linestyle=":")
plt.xlabel('Proportion water'),plt.ylabel('Posterior');

In [ ]:
# Update new prior
prior = posterior

Or, we can do it all at once, with all the data

In [ ]:
# New observations
W = 8
L = 5
# Number of trials
N = W+L

# Calculate likelihood
likelihood = dbinom(W,N,p_grid)
likelihood
# Bayes theorem
posterior = (likelihood*prior1)/sum(likelihood*prior1)
posterior

In [ ]:
# Plot posterior over range of p_grid
plt.plot(p_grid, posterior)
plt.plot(p_grid, prior1,linestyle=":")
plt.xlabel('Proportion water'),plt.ylabel('Posterior');

With this code we have built a Bayesian model to estimate the proportion of water on the earth's surface by recording the number of times our right hand lands on water versus land, given a default 'ignorant' prior. However our heterogenious educations give us some sense better than ignorance about what proportion of the earth is water. We can encode this information in a new prior and see what the data show. There are many ways to do this, but one way is to weight each value of p_grid by our sense of how likely they are.

In [ ]:
p_grid

In [ ]:
# Subjective prior
my_prior = np.array([0,0,0,0,0,0,0,0,0.02,0.05,0.10,0.15,0.20,0.20,0.15,0.10,0.05,0.02,0,0])

In [ ]:
sum(my_prior)

In [ ]:
# Standardize
my_prior = my_prior/sum(my_prior)

In [ ]:
sum(my_prior)

In [ ]:
# Plot my prior over range of p_grid
plt.plot(p_grid,my_prior)
plt.xlabel('Proportion water'),plt.ylabel('Prior');

In [ ]:
# Bayes theorem
posterior2 = (likelihood*my_prior)/sum(likelihood*my_prior)
posterior2

In [ ]:
# Plot posterior2 over range of p_grid
plt.plot(p_grid, np.ones(20)*0.05, label='Flat prior', c='dodgerblue', linestyle=":")
plt.plot(p_grid, my_prior, label='My prior', c='red', linestyle=":")
plt.plot(p_grid, posterior, label='Flat posterior', c='dodgerblue')
plt.plot(p_grid, posterior2, label='My posterior', c='red')
plt.legend()
plt.xlabel('Proportion water'),plt.ylabel('Posterior');

# Quadratic approximation

Grid approximations are helpful for learning how Bayes theorem works but they do not scale well, exponentially in fact relative to the number of parameters in the model. For most models we'll use Markov Chain Monte Carlo (MCMC) methods, but before that the quadratic approximation (QA) is a useful method to learn as well as it is really fast and forms the basis of an important MCMC alternative, the [Laplace approximation](https://bookdown.org/rdpeng/advstatcomp/laplace-approximation.html), which is used in [INLA](http://www.r-inla.org/) (with a gentle intro here: https://www.precision-analytics.ca/blog/a-gentle-inla-tutorial/).

McElreath has written his own quadratic approximation algorithm `quap()` for finding the normal peak and standard deviation and you can see that on pg. 42 of the book. In Python we can use the `find_MAP()` function in PyMC to do the same thing (`quap` uses `MAP` under the hood). MAP stands for maximum *a posteriori*, and reflects the fact that the algorithm estimates the mode of a posterior, rather than just the likelihood alone. Is is basically a fast optimization algorhithm.

In [ ]:
# Quadratic approximation using MAP in PyMC
with pm.Model() as globe_qa:
    p = pm.Uniform('p', 0, 1)
    w = pm.Binomial('w', n=N, p=p, observed=W)
    mean_q = pm.find_MAP()
    std_q = ((1/pm.find_hessian(mean_q, vars=[p]))**0.5)[0]

In [ ]:
# Mean of normal approximation
mean_q['p']

In [ ]:
# Standard deviation of normal approximation
std_q[0]

One oddity in the code above is the `pm.find_hessian` statement, if you've not seen a Hessian before, it is simply (!) a matrix of second-order partial derivatives that can be used to calculate variance/covariance. 

In [ ]:
# analytical calculation
y1 = dbinom(W,N,p_grid)
plt.plot(p_grid, y1/sum(y1), label='True posterior')

# quadratic approximation
y2 = sp.stats.norm.pdf(p_grid, mean_q['p'], std_q)
plt.plot(p_grid, y2/sum(y2), label='Quadratic approximation')
plt.legend()

plt.title('n = 9', fontsize=14)
plt.xlabel('Proportion water', fontsize=14)
plt.ylabel('Density', fontsize=14);